In [13]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# -----------------------------------------
# Daten einlesen und vorbereiten
# -----------------------------------------
url = 'https://raw.githubusercontent.com/SilasBereit/turbomachinery-pumpenanalyse/main/volume_flow_data.csv'

df = pd.read_csv(url, sep=';', usecols=[0, 1])
df.columns = ['timestamp', 'flow_rate_m3h']
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d.%m.%y %H:%M')

# Volumenstrom m³/h → m³/s
df['flow_rate_m3s'] = df['flow_rate_m3h'] / 3600

# Zeitdifferenz zwischen Messpunkten berechnen (Sekunden)
df['time_delta_s'] = df['timestamp'].diff().dt.total_seconds()
df.loc[0, 'time_delta_s'] = 0  # Erste Zeile auf 0 setzen, da kein Delta vorhanden

# -----------------------------------------
# Pumpen-Kennlinie: Förderhöhe als Funktion vom Volumenstrom
# -----------------------------------------
flow_m3h = np.array([110, 240, 300, 350, 425, 495, 555])
head_m = np.array([23, 22, 21, 20, 18, 16, 14.3])

head_interpolator = interp1d(flow_m3h, head_m, bounds_error=False, fill_value="extrapolate")
df['head_m'] = head_interpolator(df['flow_rate_m3h'])

# Konstanten
rho = 1000  # kg/m³ (Dichte Wasser)
g = 9.81    # m/s² (Erdbeschleunigung)

# -----------------------------------------
# Aufgabe 1: Energieverbrauch berechnen
# -----------------------------------------

# Hydraulische Leistung berechnen: P = rho * g * Q * H
df['P_hydraulic'] = rho * g * df['flow_rate_m3s'] * df['head_m']  # Watt

# Eingangsleistung aus Datenblatt (Impeller Ø264 mm)
flow_for_input_power = np.array([0, 120, 260, 340, 400, 460, 540])  # m³/h
input_power_kW = np.array([12.2, 15.9, 19.9, 22, 23, 24, 25])       # kW

input_power_interpolator = interp1d(flow_for_input_power, input_power_kW,
                                    bounds_error=False, fill_value="extrapolate")

df['P_input'] = input_power_interpolator(df['flow_rate_m3h']) * 1000  # W

# Energie Input pro Zeitintervall [Joule]
df['E_input'] = df['P_input'] * df['time_delta_s']

# Gesamte zugeführte Energie in kWh
total_E_input_kWh = df['E_input'].sum() / 3.6e6

print(f"Aufgabe 1 - Gesamte zugeführte Energie: {total_E_input_kWh:.2f} kWh")

# -----------------------------------------
# Aufgabe 2: Durchschnittliche Pumpen-Wirkungsgrad berechnen
# -----------------------------------------

# Hydraulische Energie pro Intervall [Joule]
df['E_hydraulic'] = df['P_hydraulic'] * df['time_delta_s']

# Gesamte hydraulische Energie in kWh
total_E_hydraulic_kWh = df['E_hydraulic'].sum() / 3.6e6

# Wirkungsgrad als Verhältnis der hydraulischen Energie zur Inputenergie
pump_efficiency = (total_E_hydraulic_kWh / total_E_input_kWh) * 100

print(f"Aufgabe 2 - Durchschnittlicher Pumpenwirkungsgrad: {pump_efficiency:.2f} %")

# -----------------------------------------
# Aufgabe 3: Nicht genutzte Energie berechnen
# -----------------------------------------

# Differenz: Eingangsenergie minus hydraulische Energie
unused_energy_kWh = total_E_input_kWh - total_E_hydraulic_kWh

print(f"Aufgabe 3 - Nicht genutzte Energie: {unused_energy_kWh:.2f} kWh")


Aufgabe 1 - Gesamte zugeführte Energie: 80.46 kWh
Aufgabe 2 - Durchschnittlicher Pumpenwirkungsgrad: 74.26 %
Aufgabe 3 - Nicht genutzte Energie: 20.71 kWh
